In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time as systime
import datetime as dtime
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [ ]:
#def normalize(data): #feature normalization
#    data = (data - data.mean()) / (data.max() - data.min())
#    return data

month_enum={1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'jun',7:'jul',8:'aug',9:'sep',10:'oct',11:'nov',12:'dec'}

In [ ]:
TRAIN_FILENAME = 'train.csv'
TEST_FILENAME = 'test.csv'

train_raw = pd.read_csv('../input/'+TRAIN_FILENAME, parse_dates=['Dates'], index_col=False)
test_raw = pd.read_csv('../input/'+TEST_FILENAME, parse_dates=['Dates'], index_col=False)

In [ ]:
def feature_engineering(data):
    
    #Get binarized weekdays, districts, and hours.
    days = pd.get_dummies(data.DayOfWeek)
    district = pd.get_dummies(data.PdDistrict)
    month = pd.get_dummies(data.Dates.dt.month.map(month_enum))
    hour = data.Dates.dt.hour
    hour = pd.get_dummies(hour) 
 
    #Append newly created dummy variables to dataframe
    new_data = pd.concat([hour, month, days, district], axis=1)
    #new_data['X']=normalize(data.X)
    #new_data['Y']=normalize(data.Y)
        
    return new_data

In [ ]:
train = feature_engineering(train_raw)
#test = pd.concat([test_raw['Id'],feature_engineering(test_raw)], axis=1)
test = feature_engineering(test_raw)

In [ ]:
cat_enc = LabelEncoder()
cat_enc.fit(train_raw['Category'])
train['CategoryEncoded'] = cat_enc.transform(train_raw['Category'])

In [ ]:
x_cols = list(train.columns[0:53].values)

In [ ]:
clf = xgb.XGBClassifier(n_estimators=5)
clf.fit(train[x_cols], train['CategoryEncoded'])
predicted = clf.predict_proba(test[x_cols])

In [ ]:
crime = cat_enc.fit_transform(train_raw.Category)
result=pd.DataFrame(predicted, columns=cat_enc.classes_)
result=result.round(4)
result= pd.concat([test_raw['Id'], result], axis=1)

result.to_csv('submit.csv', index = False)